In [1]:
import pandas as pd
# 과학적 표기 안하도록
pd.options.display.float_format = '{:.2f}'.format

import numpy as np
import datetime

---
### request+bs로 크롤링


In [2]:
import requests
from bs4 import BeautifulSoup as bs

In [42]:
# code로 접근 가능한 url
code_url = "https://finance.naver.com/item/main.nhn?code="
# 종목 분석 url
coinfo_url = "https://finance.naver.com/item/coinfo.nhn?code="
# 종목 분석 url (!naver)
coinfo_url_2 = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cmp_cd=%s&cn="
# 시가총액 목록 url
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="
size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="

---

### 시가총액 목록 사용하기

In [46]:
# 시가총액 300억 이하 목록 구하기
codeL = []

# 코스피 300억 이하 (후보)
# for i in range(19, 33):
for i in range(17, 33):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL

# 코스닥 300억 이하 (후보)
# for i in range(22, 30):
for i in range(19, 30):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
    
# 시가총액 작은 애들부터 보기
codeL.reverse()


In [ ]:
# 오늘의 시가총액 code 목록 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./"+todayDate+"_시가총액_하위권.txt", 'w')
f.write("\n".join(codeL))
f.close()

# 오늘의 시가총액 code 목록 불러오기
useDate = todayDate
# useDate = "2020_Feb_18"
f = open("./"+useDate+"_시가총액_하위권.txt", "r")
codeL = [x.replace("\n", "") for x in f.readlines()]
f.close()

---

<b>base code</b>


In [164]:
def getSoupFromCode(code):
    code_url = "https://finance.naver.com/item/main.nhn?code="
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    return soup

def cleanText(text):
    return text.replace("\n", "").replace("\t","").replace("조","").replace(",","")

def getName(soup):
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    return name

def getSoupFromReport(code):
    report_url = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cn=&cmp_cd="
    soup = bs(requests.get(report_url + code).text, 'html.parser')
    return soup

def getJsonFromConsen(code):
    consen_url = "https://navercomp.wisereport.co.kr/company/ajax/c1050001_data.aspx?flag=2&finGubun=MAIN&frq=0"+\
        "&chartType=svg&cmp_cd="
    return requests.get(consen_url + code).json()
#     json = bs(requests.get(report_url + code).text, 'html.parser')
#     return json
    

---

## 사용할 조건들

- PBR 1 미만
- PER 7 미만
- 시총 300 미만
- PCR 8 미만
- POR 8 미만


---

#### etf 인가

In [165]:
def isETF(soup):
    if(len(soup.select(".e_summary")) == 0):
        return False
    return True

#### etn 인가

In [140]:
def isETN(soup):
    name = getName(soup)
    if("ETN" in name):
        return True
    return False

---

#### 우선주인가


In [141]:
def isFirst(soup):
    name = getName(soup)
    if(name[-1] == "우"):
        return True
    if(name[-2:] in ["우A", "우B", "우C"]):
        return True
    return False

#### 시총 300억 미만인가

In [142]:
def under300(code):
    soup = getSoupFromCode(code)
    if(int(cleanText(soup.select_one("#_market_sum").text)) < 300):
        return True
    return False    

#### per 7 미만인가

In [143]:
def perUnder7(soup_report):
    try:
        per_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PER"][0]
        per_float = float(cleanText(per_str))
        if(per_float < 7 and per_float > 0):
            return per_float
        return -1
    except:
        return -1
    

#### pbr 1 미만인가

In [144]:
def pbrUnder1(soup_report):
    try:
        pbr_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PBR"][0]
        pbr_float = float(cleanText(pbr_str))
        if(pbr_float < 1 and pbr_float > 0):
            return pbr_float
        return -1
    except:
        return -1


---

#### 산업평균 per이 양수인가

In [145]:
def per_avg(soup_report):
    try:
        per_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "업종PER"][0]
        per_float = float(cleanText(per_str))
        if(per_float > 0):
            return per_float
        return -1
    except:
        return -1

In [166]:
json = getJsonFromConsen("900080")

In [168]:
json["JsonData"]

[{'YYMM': '2015.12(A)',
  'SALES': '1,973.7',
  'YOY': '12.97',
  'OP': '435.3',
  'NP': '294.8',
  'EPS': '303',
  'BPS': '3,726',
  'PER': '5.33',
  'PBR': '0.43',
  'ROE': '8.42',
  'EV': '0.04',
  'MAIN': 'IFRS연결',
  'TOT_ROW': 7},
 {'YYMM': '2016.12(A)',
  'SALES': '1,676.3',
  'YOY': '-15.07',
  'OP': '324.4',
  'NP': '279.4',
  'EPS': '271',
  'BPS': '3,755',
  'PER': '4.76',
  'PBR': '0.34',
  'ROE': '7.24',
  'EV': '-1.28',
  'MAIN': 'IFRS연결',
  'TOT_ROW': 7},
 {'YYMM': '2017.12(A)',
  'SALES': '1,572.6',
  'YOY': '-6.18',
  'OP': '294.8',
  'NP': '199.9',
  'EPS': '180',
  'BPS': '3,536',
  'PER': '5.41',
  'PBR': '0.28',
  'ROE': '4.96',
  'EV': '-1.07',
  'MAIN': 'IFRS연결',
  'TOT_ROW': 7},
 {'YYMM': '2018.12(A)',
  'SALES': '1,271.7',
  'YOY': '-19.13',
  'OP': '219.1',
  'NP': '174.0',
  'EPS': '151',
  'BPS': '3,776',
  'PER': '3.74',
  'PBR': '0.15',
  'ROE': '4.13',
  'EV': '-3.60',
  'MAIN': 'IFRS연결',
  'TOT_ROW': 7},
 {'YYMM': '2019.12(E)',
  'SALES': '',
  'YOY': '',

In [150]:
soup_report.select_one("#cTB511")

In [151]:
soup_report


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html lang="ko">
<head>
<title>온라인기업정보 - 기업모니터 - 기업개요(에스앤씨엔진그룹)</title>
<link href="/favicon.ico" rel="shortcut icon"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script language="javascript" src="/include/domain.js" type="text/javascript"></script>
<!--[if (!IE) | (gt IE 8)]>
    	<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" />
    <![endif]-->
<!--[if IE 6]>
            <link href="/include/only_ie6.css" type="text/css" rel="stylesheet"/>
        <![endif]-->
<!--[if lt IE 8]>
            <link href="/include/lt_ie8.css" type="text/css" rel="stylesheet"/>
        <![endif]-->
<link href="/v2/include/css?v=q7Pq73b9jj00gmFmOVUSxzXv_5FtLNW9qxkqFwJqh-g1" rel="stylesheet"/>
<link href="/v2/include/css5?v=pOYHbIz7ID2SE-xMWYeZW-qM3UKfSjJFh6rlqsQwj541" rel="stylesheet"/>
<script src="/v2/include/js?v=U028CNZfaQK4QM5ihH

In [134]:
soup_report.select("#cTB511")

[]

---

## 괜찮은 종목 거르기

In [126]:
report_url = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cn=&cmp_cd="


In [128]:
good = []
for code in codeL:
    
    # 자원 확보
    soup = getSoupFromCode(code)
    soup_report = getSoupFromReport(code)
    name = getName(soup)
    
    # etf/etn/우선주 제외
    if(isETF(soup) or isETN(soup) or isFirst(soup)):
        continue
        
    # 0<per<7, 0<pbr<1
    per = perUnder7(soup_report)
    pbr = pbrUnder1(soup_report)
    perAvg = per_avg(soup_report)
    if(per == -1 or pbr == -1 or perAvg == -1):
        continue
        
    # 출력
    good.append(name + " " + str(per) + " " + str(pbr))
    print(name, per, pbr)

파티게임즈 0.94 0.58
영신금속 2.87 0.39
동일철강 4.51 0.28
PN풍년 3.21 0.43
에스앤씨엔진그룹 2.1 0.05
패션플랫폼 5.59 0.57
제일테크노스 5.98 0.3
피앤텔 0.52 0.38
성우테크론 6.22 0.54
신원종합개발 1.98 0.29
원일특강 3.1 0.26
한국컴퓨터 4.23 0.32
코리아에셋투자증권 4.42 0.51
아즈텍WB 6.83 0.34
유비벨록스 5.98 0.44
DSC인베스트먼트 5.51 0.96
바이오로그디바이스 6.18 0.69
에코바이오 5.39 0.62
오리콤 6.5 0.46
파워넷 6.49 0.65
에스에이티 4.36 0.48
한일네트웍스 5.89 0.62
골든센츄리 1.09 0.14
케이디켐 5.68 0.53
슈프리마에이치큐 5.4 0.27
현대통신 4.51 0.58
메디앙스 1.24 0.59


KeyboardInterrupt: 

In [ ]:
len(good)

In [ ]:
good

---

## 나의 네이버증권 셋팅하기

In [13]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

import time

In [14]:
# 미리 로그인
driver = webdriver.Chrome('./chromedriver')
itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL"
driver.get(itemListUrl)

In [16]:
for group in range(6, 9):

    # 링크 접속 및 로그인
    itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=%s&type=OVERALL" % str(group)
    print(itemListUrl )
    
    # 기존 관심 종목 삭제
    time.sleep(0.5)
    driver.get(itemListUrl)
    time.sleep(0.5)
    driver.find_element_by_css_selector("#all_select").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("#wrap > div.section_mys > div.group_mystb.NE\=a\:lst > div:nth-child(2) > button.btn_del.NPI\=a\:del").click()
    time.sleep(0.5)
    Alert(driver).accept()
    
    # 적합 종목 리스트 찾기
    if(group == 6):
        useL = score25_code.copy()
    elif(group == 7):
        useL = score23_code.copy()
    elif(group == 8):
        useL = score20_code.copy()
            
    # 관심종목 목록 등록
    for nowCode in useL:
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").clear()
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").send_keys(nowCode)
        driver.find_element_by_css_selector("#mystock_input").send_keys(Keys.RETURN)


https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=7&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=8&type=OVERALL
